In [1]:
import cv2 , time
import numpy as np
from skimage import io
from skimage.measure import find_contours
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.draw import line
from scipy import ndimage
import playsound


In [8]:
# Calculator window
cal=np.ones((480,640), dtype = "uint8")*255
# To check if plus used before then it's equal sign
plus=0
# Holding first & second operands
first_op=[]
second_op=[]
# Holding the last number to choose to select next one or not if (transition state)
last_number = "Transition"
# Holding the numbers to be written on the screen
my_string=""
# holding the occurance of each sign between the two Transition states
occurance=np.zeros(12)
cap = cv2.VideoCapture(0)
hsvlower = np.array([0, 10, 60], dtype = "uint8") 
hsvupper = np.array([20, 150, 255], dtype = "uint8")


lower = np.array([80, 135, 85], dtype = "uint8") 
upper = np.array([255, 180, 135], dtype = "uint8")

# For transition state handling 
count =0
SE = np.ones((15,15))
SEH = np.ones((4,2))

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.medianBlur(frame,3)
    blur = cv2.GaussianBlur(frame,(3,3),3)
    imgYCC = cv2.cvtColor(blur, cv2.COLOR_BGR2YCR_CB)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv, hsvlower, hsvupper)
    mask = cv2.inRange(imgYCC, lower, upper )
    #mask = cv2.bitwise_and(mask1, mask2)
    mask = cv2.medianBlur(mask,7)
    
    # Hole filling
    height, width = mask.shape[:2]
    # Apply morphology operation to fill small holes in the image
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    # Find connected regions (to hand) to remove background objects
    # Use floodfill with a small image area (7 x 7 px) that is set gray color value
    kernel2 = 3
    mask[int(height/2)-kernel2:int(height/2)+kernel2, int(width/2)-kernel2:int(width/2)+kernel2] = 128
    # a black mask to mask the 'non-connected' components black
    balck_mask = np.zeros((height + 2, width + 2), np.uint8)
    floodImg = mask.copy()
    # Use floodFill function to paint the connected regions white 
    cv2.floodFill(floodImg, balck_mask, (int(width/2), int(height/2)), 255, flags=(4 | 255 << 8))
    # apply a binary threshold to show only connected hand region
    ret, floodedImg = cv2.threshold(floodImg, 135, 255, cv2.THRESH_BINARY)
    mask = floodedImg 
    
    ndimage.binary_fill_holes(mask, structure=np.ones((5,5))).astype(int)
    
    cnts, _ = cv2.findContours(mask.copy(),mode=2,method=1)
    cv2.drawContours(frame,cnts, -1, (0, 0, 255))
    for contour in cnts:
        area = cv2.contourArea(contour)
        #print(contour)
        if area >10000 and area <25000:
            x,y,w,h = cv2.boundingRect(contour)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            Xvalues = contour[:,0,0]
            Yvalues = contour[:,0,1]      
            Xmin = int(np.min(Xvalues))
            Xmax = int(np.max(Xvalues))
            Ymin = int(np.min(Yvalues))
            Ymax = int(np.max(Yvalues))
            
            if np.count_nonzero(mask[x:x+w,y:y+h] == 255) >500 and np.count_nonzero(mask[x:x+w,y:y+h] == 255) <15000: 
                #print(x,y)
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),5) 
                gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
                if w >100:
                    #cv2.rectangle(frame,(Xmin,Ymin),(Xmax,Ymax),(0,0,255),10) 
                    segmented = np.zeros((Ymax-Ymin,Xmax-Xmin))
                    segmented2 = np.zeros((Ymax-Ymin,Xmax-Xmin))
                    segmented = gray[Xmin:Xmax,Ymin:Ymax]
                    segmented2 = mask[Xmin:Xmax,Ymin:Ymax]
                    #segmented2=cv2.morphologyEx(segmented2, cv2.MORPH_OPEN, SE)
                    #segmented2=cv2.morphologyEx(segmented2, cv2.MORPH_CLOSE, SE)
                    
                    #cv2.floodFill(segmented2,np.zeros((segmented2.shape[0]+2,segmented2.shape[1]+2),np.uint8),(0,0),255)
                if segmented.shape[0]>0 and segmented.shape[1]>0 and segmented2.shape[0]>0 and segmented2.shape[1]>0:
                    count+=1
                    detection(mask,contour)
                    ret3,segmented = cv2.threshold(segmented,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                    
                    segmented=cv2.erode(segmented,SE,iterations = 15)
                    segmented=cv2.morphologyEx(segmented, cv2.MORPH_CLOSE, SEH)
        else:
            x,y,w,h = cv2.boundingRect(contour)
            cv2.rectangle(mask,(x,y),(x+w,y+h),(0,0,0),-1)
    
    cv2.imshow("frame", frame)
    cv2.imshow("mask", mask)
    cv2.imshow("calculator", cal)
    # Write numbers to the screen
    font = cv2.FONT_HERSHEY_SIMPLEX
    y0, dy = 50, 50
    for i, l in enumerate(my_string.split('*')):
        y = y0 + i*dy
        cv2.putText(cal, l, (10, y ), font, 1, 2)
    # observe the keypress by the user
    keypress = cv2.waitKey(1) & 0xFF

    # if the user pressed "q", then stop looping
    if keypress == ord("q"):
        break

# free up memory
cap.release()
cv2.destroyAllWindows()

In [2]:
def convex_hull(points):
    # Sort the points lexicographically (tuples are compared lexicographically).
    # Remove duplicates to detect the case we have just one unique point.
    points = sorted(set(points),key=lambda x: (x[1], x[0]))
    # Boring case: no points or a single point, possibly repeated multiple times.
    if len(points) <= 1:
        return points

    # 2D cross product of OA and OB vectors, i.e. z-component of their 3D cross product.
    # Returns a positive value, if OAB makes a counter-clockwise turn,
    # negative for clockwise turn, and zero if the points are collinear.
    def cross(o, a, b):
        return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])

    indices_of_hulls_lower = []
    # Build lower hull 
    lower = []
    for i in range(len(points)):
        while len(lower) >= 2 and cross(lower[-2], lower[-1], points[i]) <= 0:
            lower.pop()
            indices_of_hulls_lower.pop()
        lower.append(points[i])
        indices_of_hulls_lower.append(i)

    indices_of_hulls_upper = []
    # Build upper hull
    upper = []
    for i in range(len(points)-1,0,-1):
        while len(upper) >= 2 and cross(upper[-2], upper[-1], points[i]) <= 0:
            upper.pop()
            indices_of_hulls_upper.pop()
        upper.append(points[i])
        indices_of_hulls_upper.append(i)

    # Concatenation of the lower and upper hulls gives the convex hull.
    # Delete the last point of upper and lower hull as it exists in both the arrays
    return lower[:-1] , upper[:-1] , indices_of_hulls_lower[:-1],indices_of_hulls_upper[:-1], points


In [3]:
# Function to get defects
def convexity_defects(hulls,indices_of_hulls,contour):
    # contains the defects
    defects=[]
    for i in range(len(hulls)-1):
            maximum=-1
            maximum_point=(-1,-1)
            mid_point=((hulls[i][0]+hulls[i+1][0])//2 -150,(hulls[i][1]+hulls[i+1][1])//2)
            point=(-1,-1)
            for j in range (indices_of_hulls[i],indices_of_hulls[i+1],1):
                if point[1] != contour[j][1]:
                    point=contour[j]
                    distance=np.sqrt( pow(mid_point[0]-point[0],2) + pow(mid_point[1]-point[1],2))
                    # Make the defect the farthest point from contours
                    if distance > maximum:
                        maximum=distance
                        maximum_point=point
            # Adding defects to the array of defects
            dist_first_pnt = np.sqrt( pow(hulls[i][0]-maximum_point[0],2) + pow(hulls[i][1]-maximum_point[1],2))
            dist_second_pnt = np.sqrt( pow(hulls[i+1][0]-maximum_point[0],2) + pow(hulls[i+1][1]-maximum_point[1],2))
            if maximum > -1 and (dist_first_pnt > 25 and dist_second_pnt > 25):
                defects.append(maximum_point)
    return defects

In [4]:
def detection(card,contours):  
    '''
    card=io.imread('hand\\Mahmoud\\seven.jpg')
    #card=cv2.flip(card,1)
    card = cv2.cvtColor(card, cv2.COLOR_BGR2GRAY)
    # Resize image
    scale_percent = 60 # Percent of original size
    width = int(card.shape[1] * scale_percent / 100)
    height = int(card.shape[0] * scale_percent / 100)
    dim = (429, 429)
    card = cv2.resize(card, dim, interpolation = cv2.INTER_AREA)
    # Convert image to binary
    ret, threshold = cv2.threshold(card, 200, 255, cv2.THRESH_BINARY)
    # Find all contours of the shape
    '''
    global count,occurance
    
    #contours, hiearchy = cv2.findContours(card,mode=2,method=1)
    tempcontours=contours
    #print(len(contours))
    # Contour area
    areacnt = cv2.contourArea(contours)
    # Make contours tuples 
    cnt1 = contours[:,0,0] 
    cnt2 = contours[:,0,1] 
    cnt = [(cnt2[i], cnt1[i]) for i in range(0, len(cnt2))]

    # Return contours arranged from left to right
    lower_hull , upper_hull , indices_of_lower_hulls , indices_of_upper_hulls , contours = convex_hull ( cnt )

    # Append lowerhull + upperhull
    hull = lower_hull + upper_hull

    # To reshape hulls to be able to get its area
    arrhull=np.array([0,0])
    result=np.array([0,0])
    l=0
    for i in hull:
        row_to_be_added = np.array([i[0], i[1]]) 
        # Adding row to numpy array 
        if l != 0:
            result = np.vstack ((arrhull, row_to_be_added) ) 
            arrhull=result
        else:
            arrhull=row_to_be_added
        l+=1

    # Area of the hull
    areahull = cv2.contourArea(arrhull)

    # Ratio between contour area and hull area
    ratio = areacnt/areahull

    #draw lines between hulls
    for i in range (len(hull)-1):
        rr, cc = line(int(hull[i][0]),int(hull[i][1]), int(hull[i+1][0]),int(hull[i+1][1]))
        card[rr,cc]=255

    stacked_img = np.stack((card,)*3, axis=-1)

    #New hulls including the last point
    new_hulls=[]
    new_ind=[]
    # Reverse to start from left to right
    upper_hull.reverse()
    indices_of_upper_hulls.reverse()
    #making hulls to be sent to convexity defects to calculate defects
    for i in range(len(upper_hull)-1):
        if np.sqrt(pow(upper_hull[i][0]-upper_hull[i+1][0],2)+pow(upper_hull[i][1]-upper_hull[i+1][1],2)) > 30:
            if upper_hull[i][0]-upper_hull[i+1][0] < 150:
                new_hulls.append(upper_hull[i])
                new_ind.append(indices_of_upper_hulls[i])

    #if len(new_hulls)>0 and upper_hull[-1][0]-new_hulls[-1][0] < 130:       
    new_hulls.append(upper_hull[-1])
    new_ind.append(indices_of_upper_hulls[-1])

    #Defects to be draw in the img
    defects=convexity_defects(new_hulls,new_ind,contours)
    
    integer=GetNumber(len(tempcontours),len(new_hulls),len(defects),ratio)
    txt_number=GettxtNumber(integer)
    if integer != None:
        occurance[integer]+=1
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,txt_number,(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    
   
    if txt_number == "Transition" and count> 30:
        num=0
        for i in range(len(occurance)-1):
            if occurance[num]<occurance[i]:
                num=i
        # To pass number only when the last sign is transition state and afer 60 frame to make sure the sign is true
        if occurance[num]>10:
            if(num != 10 and num != 11 and num != 0):
                playsound.playsound(str(num)+'.mp3',False)
            occurance[:]=0
            calculator(GettxtNumber(num),num)
            count=0
        
    for i in defects:
        frame[int(i[0]):int(i[0]+15),int(i[1]):int(i[1]+15)]=(0,0,255)

    for i in new_hulls:
        frame[int(i[0]):int(i[0]+15),int(i[1]):int(i[1]+15)]=(255,0,0)
    
    
'''
    plt.figure(figsize=(8,8))

    io.imshow(stacked_img)
    io.show()

    print(GetNumber(len(tempcontours),len(new_hulls),len(defects),ratio))

    print(len(defects),len(new_hulls))
'''

'\n    plt.figure(figsize=(8,8))\n\n    io.imshow(stacked_img)\n    io.show()\n\n    print(GetNumber(len(tempcontours),len(new_hulls),len(defects),ratio))\n\n    print(len(defects),len(new_hulls))\n'

In [5]:
def GetNumber(contourscnt,hulls,defects,ratio):
    xdif=Xmax-Xmin
    ydif=Ymax-Ymin
    font = cv2.FONT_HERSHEY_SIMPLEX
    lengthbetweendiagonal = np.sqrt( pow(xdif,2) + pow(ydif,2))
    if ratio >.88 and ratio <.96:
        if (xdif/ydif)<1.25:
            if (xdif/ydif)>0.67:
                return 11
            else:
                return 9
        else :
            if (xdif/ydif)<3: 
                return 10
            else:
                return 0
    elif defects ==0:
        if (xdif/ydif)<0.6:
            return 1
        elif ratio>0.6 and ratio <0.9 :
            if (xdif/ydif)<1.3 :
                return 6
            else:
                if (xdif/ydif)<2:
                    return 8
                else:
                    return 0
    elif defects==1:
        if (xdif/ydif)>0.6:
            return 7
        else:
            return 2
    elif defects == 2:
        return 3
    elif defects == 3:
        return 4
    elif defects == 4:
        return 5

In [6]:
def GettxtNumber(number):

    if number ==0:
        return 'Zero'
    elif number == 1:
        return 'One'
    elif number ==2:
        return 'Two'
    elif number == 3:
        return 'Three'
    elif number == 4:
        return 'Four'
    elif number == 5:
        return 'Five'
    elif number == 6:
        return 'Six'
    elif number == 7:
        return 'Seven'
    elif number == 8:
        return 'Eight'
    elif number == 9:
        return 'Nine'
    elif number == 10:
        return 'Plus'
    elif number == 11:
        return 'Transition'

In [7]:
def calculator(sign,number):
    global plus,first_op,second_op,my_string
    if plus < 2:
        if sign == None or number == None:
            return
        last_number=number
        if sign == "Plus" and plus == 0:
            my_string+="+"
        elif sign == "Plus" and plus:
            my_string+="="
        elif plus == True:
            second_op.append(number)
            my_string+=str(number)
        else:
            first_op.append(number)
            my_string+=str(number)
        if plus == True and sign == "Plus":
            plus += 1
            num=0
            for i in range(len(first_op)):
                num+=first_op[i]*pow(10,len(first_op)-1 - i)
            for i in range(len(second_op)):
                num+=second_op[i]*pow(10,len(second_op)-1 - i)
            st_len=len(my_string)//28
            if (len(my_string)+len(str(num)))//28 > st_len:
                my_string+='*'
            my_string+=str(num)
            
        if sign == "Plus":
            plus += 1
        if len(my_string)%28 == 0:
            my_string+='*'